In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Importing Libraries

In [3]:
#statistics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16

#data structures
import numpy as np

#visualization
import matplotlib.pyplot as plt

#other
import os

print(tf.__version__) #1.14
print(keras.__version__) #2.2.4

1.14.0
2.2.4-tf


In [4]:
tf.enable_eager_execution()

# Create Model

In [67]:
OUTPUT_CHANNELS = 1

In [88]:
def frontend_feat(filters, size, apply_batchnorm=True):
    '''
    objective: create frontend parts
    return: result - keras.Sequential
    '''
    initializer = tf.random_normal_initializer(0., 0.02)
    result = keras.Sequential()
    
    if filters == 'M':
        result.add(
            keras.layers.MaxPooling2D())
    else:
        result.add(
          keras.layers.Conv2D(filters, size, padding='same',
                                 kernel_initializer=initializer, use_bias=False))

        if apply_batchnorm:
            result.add(keras.layers.BatchNormalization())

        result.add(keras.layers.ReLU())

    return result

In [168]:
def backend_feat(filters, size, apply_dropout=False):
    '''
    objective: create backend parts
    return: result - keras.Sequential
    '''
    initializer = tf.random_normal_initializer(0., 0.02)

    result = keras.Sequential()
    result.add(
    keras.layers.Conv2DTranspose(filters, size, padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

    result.add(keras.layers.BatchNormalization())

    if apply_dropout:
        result.add(keras.layers.Dropout(0.5))

    result.add(keras.layers.ReLU())

    return result

In [175]:
def create_model():
    '''
    objective: create full model
    return: result - keras.Model
    '''
    frontend_stack = [
        frontend_feat(64, 3, apply_batchnorm= False),
        frontend_feat(64, 3),
        frontend_feat('M', 2),
        frontend_feat(128, 3),
        frontend_feat(128, 3),
        frontend_feat('M', 2),
        frontend_feat(256, 3),
        frontend_feat(256, 3),
        frontend_feat(256, 3),
        frontend_feat('M', 2),
        frontend_feat(512, 3),
        frontend_feat(512, 3),
        frontend_feat(512, 3),
    ]    
    backend_stack = [
        backend_feat(512, 3),
        backend_feat(512, 3),
        backend_feat(512, 3),
        backend_feat(256, 3),
        backend_feat(128, 3),
        backend_feat(64, 3),
    ]
    
    initializer = tf.random_normal_initializer(0., 0.02)
    ######
    last_layer = keras.layers.Dense(OUTPUT_CHANNELS)
    concat = keras.layers.Concatenate()
    inputs = keras.layers.Input(shape = (None,None,3))
    x = inputs
    
    skips = []
    for front in frontend_stack:
        x = front(x)
        skips.append(x)
        
    skips = reversed(skips[:-1])
    
    for back, skip in zip(backend_stack, skips):
        x = back(x)
        x = concat([x, skip])
        
    x = last_layer(x)
    
    return keras.Model(inputs = inputs, outputs = x)

## check if it's created well

In [176]:
def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image, channels = 3)
    #image = tf.squeeze(image)
    image = tf.image.resize(image, (255,255))

    input_image = image

    input_image = tf.cast(input_image, tf.float32)

    return input_image

In [177]:
inp = load('ShanghaiTech/part_A/train_data/images/IMG_100.jpg')

In [178]:
down_model = frontend_feat(64, 3)
down_result = down_model(tf.expand_dims(inp, 0))
print (down_result.shape)

(1, 255, 255, 64)


In [179]:
up_model = backend_feat(64,3)
up_result = up_model(down_result)
print(up_result.shape)

(1, 255, 255, 64)


In [181]:
generator = create_model()

gen_output = generator(tf.expand_dims(inp, 0), training=False)
print(np.sum(gen_output))

28148.678


In [103]:
generator.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
sequential_117 (Sequential)     (None, None, None, 6 1728        input_6[0][0]                    
__________________________________________________________________________________________________
sequential_118 (Sequential)     (None, None, None, 6 37120       sequential_117[0][0]             
__________________________________________________________________________________________________
sequential_119 (Sequential)     (None, None, None, 6 0           sequential_118[0][0]             
____________________________________________________________________________________________